In [ ]:
# created with love in cooperation with ASPIS
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input, Conv1D, LSTM, MaxPooling1D, Flatten, concatenate, TimeDistributed, Bidirectional, ConvLSTM2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error
import os
import numpy as np

# Load data

In [2]:
f_name = "full-not-divided/shift-15-windows-45"

In [3]:
X_train = np.load(f'../data-preparation/{f_name}/X_train.npy',allow_pickle=True).item()
y_train = np.load(f'../data-preparation/{f_name}/y_train.npy')

In [4]:
Phi60_Sig1 = X_train["Phi60_Sig1"]

# Model

In [5]:
inputs_Phi60_Sig1 = Input(shape=(Phi60_Sig1.shape[1],1))

a = Conv1D(128, kernel_size = 2, padding = "same")(inputs_Phi60_Sig1)
a = Bidirectional(LSTM(128, dropout=0.1,recurrent_dropout=0.1))(a)
a = Dense(64, activation='relu')(a)
a = Dropout(0.2)(a)
a = Dense(64, activation='relu')(a)
a = Dense(32, activation='relu')(a)

output = Dense(1, activation='linear')(a)

model = Model(inputs=inputs_Phi60_Sig1, outputs=output)
model.compile(loss='mse', optimizer='adam', metrics=["mae"])
print(model.summary())

2023-01-03 17:08:43.320272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6674 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:8b:00.0, compute capability: 7.5


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 45, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 45, 128)           384       
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                             

In [6]:
# callbacks
os.makedirs(f_name, exist_ok=True)
saved_model = "/epoch-{epoch:02d}-mae-{val_mae:.4f}.hdf5"
checkpoint = ModelCheckpoint(f_name+"/autoreg"+saved_model, monitor='val_mae', verbose=1, save_freq="epoch")
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(
    x=Phi60_Sig1, 
    y=y_train, 
    validation_split=0.1, 
    epochs=5, 
    verbose=1, 
    callbacks=callbacks_list,
    shuffle=True
)

Epoch 1/5


2023-01-03 17:09:27.423156: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


10419/45871 [=====>........................] - ETA: 4:25:19 - loss: 0.0149 - mae: 0.0660